**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers import Flatten
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [3]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [4]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The purpose of the `act` method is to make a decision based on a state `s`. To this end, he follows a policy (expressed here by `learned_act`) that leaves room for the agent to "explore". Specifically, if the parameter `train` is set to `True`, then the agent has a probability $\epsilon$ of making a random move. To picture it, we can think of $\epsilon$ as getting the Agent out of its comfort zone. This parameter is crucial and needs careful tuning: 
- If set too high, he will get stuck and stop learning
- If set too low, the agent will systematically behave randomly

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [5]:
class Environment(object):

    def __init__(self, grid_size = 10, max_time = 500, temperature = 0.1):

        # Hyperparameters
        self.grid_size = grid_size + 4
        self.max_time = max_time
        self.temperature = temperature

        # Key trackers of the Environment
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # Coordinate of the agent
        self.x = 0
        self.y = 1

        # Time
        self.time = 0

        # Dimensioning
        self.scale = 16
        self.to_draw = np.zeros((max_time + 2, grid_size * self.scale, grid_size * self.scale, 3))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size, self.grid_size, 3))+128
        b[self.board > 0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x, self.y, :] = 256
        b[-2:, :, :]=0
        b[:, -2:, :]=0
        b[:2, :, :]=0
        b[:, :2, :]=0

        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b

    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.time))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.time = self.time + 1

        reward = self.board[self.x, self.y]

        self.board[self.x, self.y] = 0
        game_over = self.time > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size, 1),
                                self.position.reshape(self.grid_size, self.grid_size, 1)),
                                axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.time = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


The following elements are important because they correspond to the hyper parameters for this project:

In [10]:
# parameters
size = 13
T = 500
temperature = 0.3
epochs_train = 20 # set small when debugging
epochs_test = 5 # set small when debugging
movie_ = True

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

- `position` is an array (padded by -1 to indicate illegal moves) which tracks the `Agent` position (with a $1$ otherwise $0$). It looks something like this:

|  P |  O |  S |  I |  T |  I |  O | N  |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 |
| -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 |
| -1 | -1 |  0 |  0 |  0 |  0 | -1 | -1 |
| -1 | -1 |  0 |  1 |  0 |  0 | -1 | -1 |
| -1 | -1 |  0 |  0 |  0 |  0 | -1 | -1 |
| -1 | -1 |  0 |  0 |  0 |  0 | -1 | -1 |
| -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 |
| -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 |


- `board` accounts for the `Environment` as it contains the rewards (positives AND negatives) in its cells. Whenever the `Agent` reach a cell, this cell becomes `0` in `board`.

|  - |  B |  O  |  A |  R  |  D |  ! |  - |
|:--:|:--:|:---:|:--:|:---:|:--:|:--:|:--:|
| -1 | -1 |  -1 | -1 |  -1 | -1 | -1 | -1 |
| -1 | -1 |  -1 | -1 |  -1 | -1 | -1 | -1 |
| -1 | -1 |  0  |  0 |  0  |  0 | -1 | -1 |
| -1 | -1 |  0  |  0 |  -1 |  0 | -1 | -1 |
| -1 | -1 | 0.5 |  0 | 0.5 |  0 | -1 | -1 |
| -1 | -1 |  0  | -1 |  0  |  0 | -1 | -1 |
| -1 | -1 |  -1 | -1 | -1  | -1 | -1 | -1 |
| -1 | -1 |  -1 | -1 |  -1 | -1 | -1 | -1 |

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [11]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return(np.random.randint(4))

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [12]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        
        # This assumes that the games will end
        win, lose = 0, 0
        game_over = False

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win += reward
            if reward < 0:
                lose -= reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
                
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [13]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()
test(agent,env,epochs_test,prefix='random')

Win/lose count 14.0/17.0. Average score (-3.0)
Win/lose count 18.0/17.0. Average score (-1.0)
Win/lose count 14.5/17.0. Average score (-1.5)
Win/lose count 13.5/20.0. Average score (-2.75)
Win/lose count 20.0/30.0. Average score (-4.2)
Final score: -4.2


In [ ]:
if movie_: HTML(display_videos('random2.mp4'))

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




By definition, the $Q$ function is $\mathcal{Q}^{\pi}(s,a)=E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s, a_{0}=a]$

Therefore, by dropping the first term of the sum and using the linearity of the expectation:
$$
\begin{align}
\mathcal{Q}^{\pi}(s,a)&=E_{p^{\pi}}\big[r(s_0,a_0)+\sum_{t=1}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a\big]\\
&=E_{p^{\pi}}[r(s_0,a_0)+\gamma\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1}|s_{0}=s,a_{0}=a)]\\
&=E_{p^{\pi}}\big[r(s_0,a_0)]+\gamma E_{p^{\pi}}\big[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a\big]\\
\end{align}
$$

Then, because of the Markovian property, we obtain:

$$
\begin{align}
\mathcal{Q}^{\pi}(s,a)&=E_{p^{\pi}}\big[r(s_{0},a_{0})\big]+\gamma E_{(s',a')~p(.|s,a)}\big[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1}|s_{1}=s',a_{1}=a')\big]\\
&=E_{(s',a')\sim p(.|s,a)}\big[r(s_{0},a_{0})\big]+\gamma E_{(s',a')\sim p(.|s,a)}\big[\sum_{t=0}^{\infty}
\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a'\big]\\
&=E_{(s',a')\sim p(.|s,a)}\big[r(s,a)+\gamma\mathcal{Q}^{\pi}(s',a')\big]\\
\end{align}
$$

If an optimal policy $\pi^{\ast}$ exists, the optimal $Q$ function is then defined as:

$$
\begin{align}
\mathcal{Q}^{\ast}(s,a)&=\text{max}_{\pi}\mathcal{Q}^{\pi}(s,a)\\
&=\text{max}_{\pi}E_{(s',a')\sim p(.|s,a)}\big[r(s,a)+\gamma\mathcal{Q}^{\pi}(s',a')\big]\\
&=E_{s'\sim\pi^{\ast}(.|s,a)}\big[r(s,a)+\gamma\mathcal{Q}^{\pi^{\ast}}(s',a')\big]\\
&=E_{s'\sim\pi^{\ast}(.|s,a)}\big[r(s,a)+\gamma \text{max}_{a'}\mathcal{Q}^{\ast}(s',a')\big]
\end{align}
$$

Combining these equations and because of the deterministic nature of $r$, a plausible loss function to minimize becomes:

$$
\begin{align}
\mathcal{L}(\theta)&=E_{s'\sim\pi^{\ast}(.|s,a)}\big[(\mathcal{Q}^{\ast}(s,a,\theta)-\mathcal{Q}(s,a,\theta))^{2}\big]\\
&=E_{s'\sim\pi^{\ast}(.|s,a)}\big[(r(s,a)+\gamma \text{max}_{a'}\mathcal{Q}^{\ast}(s',a',\theta)-\mathcal{Q}(s,a,\theta))^{2}\big]\\
&=E_{s'\sim\pi^{\ast}(.|s,a)}\big[||r+\gamma {max}_{a'}\mathcal{Q}(s',a',\theta)-\mathcal{Q}(s,a,\theta)||^{2}\big]
\end{align}
$$

which is the desired result.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [22]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory: self.memory.pop(0)

    def random_access(self):
        return(self.memory[np.random.randint(len(self.memory))])

***
The pipeline we will use for training is given below:

In [23]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent follows his policy
            action = agent.act(state)

            # Environment update
            previous_state = state
            state, reward, game_over = env.act(action)

            # Update key variables
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(previous_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [25]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.9
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        prediction=self.model.predict(np.array([s,]))
        return np.argmax(prediction)

    def reinforce(self, s_, n_s_, a_, r_, game_over):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            # Random Access Memories
            [s_batch, n_s_batch, a_batch, r_batch, game_over_batch] = self.memory.random_access()
            input_states[i] = s_batch
            
            # Model prediction
            target_q[i] = self.model.predict(np.array([s_batch]))

            # Compute the Q values targets if the game is not over
            if game_over:
                target_q[i, a_batch] = r_batch
            else:
                prediction = self.model.predict(np.array([n_s_batch]))
                target_q[i, a_batch] = r_batch + self.discount * np.amax(prediction)

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        return self.model.train_on_batch(input_states, target_q)

    def save(self, name_weights='model.h5', name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self, name_weights='model.h5', name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
            
class DQN_FC(DQN):
    
    def __init__(self, *args, lr = 0.1, **kwargs):
        super(DQN_FC, self).__init__(*args, **kwargs)
        
        # Fully connected model
        model = Sequential()

        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(32))
        model.add(Activation('relu'))

        model.add(Dense(32))
        model.add(Activation('relu'))

        model.add(Dense(4))
        model.add(Activation('linear'))

        model.compile(sgd(lr = lr, decay = 1e-4, momentum = 0.8), "mse")
        self.model = model

In [26]:
# Instanciate environment
env_fc_basic = Environment(grid_size = size, max_time = T, temperature = 0.3)
agent_fc_basic = DQN_FC(size, lr = 0.1, epsilon = 0.1, memory_size = 2000, batch_size = 32)
train(agent_fc_basic, env_fc_basic, 40, prefix = 'fc_train_basic_')

Epoch 000/040 | Loss 0.0108 | Win/lose count 3.0/10.0 (-7.0)
Epoch 001/040 | Loss 0.0126 | Win/lose count 6.0/10.0 (-4.0)
Epoch 002/040 | Loss 0.0305 | Win/lose count 9.0/10.0 (-1.0)
Epoch 003/040 | Loss 0.0400 | Win/lose count 3.5/2.0 (1.5)
Epoch 004/040 | Loss 0.0673 | Win/lose count 9.0/3.0 (6.0)
Epoch 005/040 | Loss 0.1240 | Win/lose count 4.5/2.0 (2.5)
Epoch 006/040 | Loss 0.0698 | Win/lose count 5.5/6.0 (-0.5)
Epoch 007/040 | Loss 0.0971 | Win/lose count 13.5/8.0 (5.5)
Epoch 008/040 | Loss 0.0848 | Win/lose count 11.5/8.0 (3.5)
Epoch 009/040 | Loss 0.0983 | Win/lose count 11.5/10.0 (1.5)
Epoch 010/040 | Loss 0.0952 | Win/lose count 9.0/3.0 (6.0)
Epoch 011/040 | Loss 0.1144 | Win/lose count 8.0/6.0 (2.0)
Epoch 012/040 | Loss 0.0991 | Win/lose count 20.0/5.0 (15.0)
Epoch 013/040 | Loss 0.1250 | Win/lose count 7.5/6.0 (1.5)
Epoch 014/040 | Loss 0.1025 | Win/lose count 4.5/4.0 (0.5)
Epoch 015/040 | Loss 0.1550 | Win/lose count 13.0/3.0 (10.0)
Epoch 016/040 | Loss 0.1538 | Win/lose co

In [27]:
if movie_: HTML(display_videos('fc_train_basic_30.mp4'))

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [28]:
class DQN_CNN(DQN):

    def __init__(self, *args, lr = 0.1, **kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        # CNN model
        model = Sequential() 
        # Strides are due to the multiple images to represent the game
        model.add(Conv2D(32, kernel_size = (1, 1), strides = (1, 1), input_shape = (5, 5, self.n_state), data_format = 'channels_last'))
        model.add(Activation('relu'))
        
        model.add(Conv2D(16, kernel_size = (3, 3), strides = (1, 1), data_format = 'channels_last'))
        model.add(Activation('relu'))
        
        model.add(Flatten())
        model.add(Dense(4)) 
        
        model.add(Activation('linear')) 
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.8), "mse")
        self.model = model

In [29]:
# Set environment with by default temperature
env_cnn_basic = Environment(grid_size = size, max_time = T, temperature = 0.3)
# Instanciate agent
agent_cnn_basic = DQN_CNN(size, lr = 0.1, epsilon = 0.2, memory_size = 2000, batch_size = 32)
# Train + test
train(agent_cnn_basic, env_cnn_basic, 40, prefix='cnn_train_basic_')

Epoch 000/040 | Loss 0.3766 | Win/lose count 27.0/12.0 (15.0)
Epoch 001/040 | Loss 0.2782 | Win/lose count 22.0/11.0 (11.0)
Epoch 002/040 | Loss 0.4457 | Win/lose count 21.5/14.0 (7.5)
Epoch 003/040 | Loss 0.3885 | Win/lose count 22.5/14.0 (8.5)
Epoch 004/040 | Loss 0.2071 | Win/lose count 24.0/11.0 (13.0)
Epoch 005/040 | Loss 0.2117 | Win/lose count 22.0/12.0 (10.0)
Epoch 006/040 | Loss 0.2471 | Win/lose count 23.0/10.0 (13.0)
Epoch 007/040 | Loss 0.2187 | Win/lose count 18.0/13.0 (5.0)
Epoch 008/040 | Loss 0.2164 | Win/lose count 17.0/14.0 (3.0)
Epoch 009/040 | Loss 0.1843 | Win/lose count 27.0/10.0 (17.0)
Epoch 010/040 | Loss 0.3088 | Win/lose count 22.5/12.0 (10.5)
Epoch 011/040 | Loss 0.2695 | Win/lose count 26.0/10.0 (16.0)
Epoch 012/040 | Loss 0.2692 | Win/lose count 18.5/9.0 (9.5)
Epoch 013/040 | Loss 0.2847 | Win/lose count 27.0/11.0 (16.0)
Epoch 014/040 | Loss 0.3416 | Win/lose count 30.0/9.0 (21.0)
Epoch 015/040 | Loss 0.2026 | Win/lose count 23.0/9.0 (14.0)
Epoch 016/040 | 

In [35]:
#if movie_: 
HTML(display_videos('cnn_train_basic_30.mp4'))

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

> Test CNN (w/o high temperature)

In [36]:
print('Test basic CNN:')
# Set environment with by default temperature
env_cnn = Environment(grid_size = size, max_time = T, temperature = 0.3)
# Instanciate CNN agent with trained weights
agent_cnn = DQN_CNN(size, lr = 0.1, epsilon = 0.2, memory_size = 2000, batch_size = 32)
agent_cnn.load(name_weights = 'cnn_train_basic_model.h5', name_model = 'cnn_train_basic_model.json')
# Test agent
test(agent_cnn, env_cnn, 5, prefix = 'cnn_test_basic_')

Test basic CNN:
Win/lose count 27.0/15.0. Average score (12.0)
Win/lose count 26.0/11.0. Average score (13.5)
Win/lose count 20.0/5.0. Average score (14.0)
Win/lose count 17.0/7.0. Average score (13.0)
Win/lose count 17.5/3.0. Average score (13.3)
Final score: 13.3


In [32]:
if movie_: HTML(display_videos('cnn_test_basic_4.mp4'))

In [34]:
print('Train CNN with higher temperature:')
# Set environment with by doubled temperature
env_temp = Environment(grid_size = size, max_time = T, temperature = 0.6)
# Instanciate agent
agent_temp = DQN_CNN(size, lr =0.1, epsilon = 0.2, memory_size = 2000, batch_size = 32)
# Train + test
train(agent_temp, env_temp, 40, prefix = 'cnn_train_temperature_')

Train CNN with higher temperature:
Epoch 000/040 | Loss 0.6633 | Win/lose count 35.0/24.0 (11.0)
Epoch 001/040 | Loss 0.8482 | Win/lose count 45.5/19.0 (26.5)
Epoch 002/040 | Loss 0.8847 | Win/lose count 47.5/11.0 (36.5)
Epoch 003/040 | Loss 0.5738 | Win/lose count 46.0/16.0 (30.0)
Epoch 004/040 | Loss 0.6070 | Win/lose count 32.0/10.0 (22.0)
Epoch 005/040 | Loss 0.4540 | Win/lose count 43.5/12.0 (31.5)
Epoch 006/040 | Loss 0.4505 | Win/lose count 46.0/11.0 (35.0)
Epoch 007/040 | Loss 0.5240 | Win/lose count 45.5/23.0 (22.5)
Epoch 008/040 | Loss 0.5002 | Win/lose count 50.0/16.0 (34.0)
Epoch 009/040 | Loss 0.4215 | Win/lose count 44.5/11.0 (33.5)
Epoch 010/040 | Loss 0.4185 | Win/lose count 45.5/14.0 (31.5)
Epoch 011/040 | Loss 0.4039 | Win/lose count 52.5/19.0 (33.5)
Epoch 012/040 | Loss 0.4170 | Win/lose count 51.5/14.0 (37.5)
Epoch 013/040 | Loss 0.4883 | Win/lose count 45.5/16.0 (29.5)
Epoch 014/040 | Loss 0.3561 | Win/lose count 42.0/8.0 (34.0)
Epoch 015/040 | Loss 0.5052 | Win/lo

In [ ]:
#if movie_: 
HTML(display_videos('cnn_train_temperature_0.mp4'))

In [38]:
print('Test CNN with higher temperature:')
# Set environment with by doubled temperature
env_cnn_temp = Environment(grid_size = size, max_time = T, temperature = 0.6)
# Instanciate CNN agent
agent_cnn_temp = DQN_CNN(size, lr = 0.1, epsilon = 0.2, memory_size = 2000, batch_size = 32)
agent_cnn_temp.load(name_weights = 'cnn_train_temperature_model.h5', name_model = 'cnn_train_temperature_model.json')
test(agent_cnn_temp, env_cnn_temp, 5, prefix = 'cnn_test_temperature_')

Test CNN with higher temperature:
Win/lose count 40.0/15.0. Average score (25.0)
Win/lose count 36.0/12.0. Average score (24.5)
Win/lose count 46.0/16.0. Average score (26.333333333333332)
Win/lose count 48.0/11.0. Average score (29.0)
Win/lose count 42.0/14.0. Average score (28.8)
Final score: 28.8


In [39]:
#if movie_: 
HTML(display_videos('cnn_test_temperature_4.mp4'))

> Test Fully Connected (w/o high temperature)

In [ ]:
print('Test basic FC:')
# Set environment with by default temperature
env_cnn = Environment(grid_size = size, max_time = T, temperature = 0.3)
# Instanciate CNN agent with trained weights
agent_fc = DQN_FC(size, lr = 0.1, epsilon = 0.2, memory_size = 2000, batch_size = 32)
agent_fc.load(name_weights = 'fc_train_basic_model.h5', name_model = 'fc_train_basic_model.json')
# Test agent
test(agent_fc, env_fc, epochs_test, prefix = 'fc_test_basic_')

In [ ]:
if movie_: HTML(display_videos('fc_test_basic_9.mp4'))

In [ ]:
print('Train FC with higher temperature:')
# Set environment with by doubled temperature
env_temp = Environment(grid_size = size, max_time = T, temperature = 0.6)
# Instanciate agent
agent_temp = DQN_FC(size, lr = 0.1, epsilon = 0.2, memory_size = 2000, batch_size = 32)
# Train + test
train(agent_temp, env_temp, epochs_train,prefix = 'fc_train_temperature_')

In [ ]:
if movie_: HTML(display_videos('fc_train_temperature_40.mp4'))

In [ ]:
print('Test of the FC with higher temperature')
# Set environment with by doubled temperature
env_fc_temp = Environment(grid_size = size, max_time = T, temperature = 0.6)
# Instanciate FC agent
agent_fc_temp = DQN_FC(size, lr = 0.1, epsilon = 0.2, memory_size = 2000, batch_size = 32)
agent_fc_temp.load(name_weights = 'fc_train_temperature_model.h5', name_model = 'fc_train_temperature_model.json')
# Test
test(agent_fc_temp, env_fc_temp, epochs_test, prefix = 'fc_test_temperature_')
HTML(display_videos('fc_test_temperature_40.mp4'))

In [ ]:
if movie_: HTML(display_videos('fc_test_temperature_40.mp4'))

<b>Remarks on experiments:</b>
- CNN produce better and smoother results than FC does (and seems less "agitated")<br>
- `temperature` controls the relative amount of positive rewards within the grid. In consequence, we choose the double of the default value `temperature = 0.6` .  It seems to encourage the `Agent` to move more on the grid.<br>
- Empirically, CNN tend to get stuck when the closest positive reward is far.

<i>Note:</i> I did not have time to run the FC test. The code work nonetheless.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [40]:
def train_explore(agent, env, epoch, prefix='', epsilon_0 = 0.6):
    # Number of won games
    score, loss = 0, 0
    
    for e in range(epoch):
        # Set a decaying epsilon
        agent.set_epsilon(epsilon_0 - (epsilon_0 - 0.1)*e/epoch)

        # Reset the environment for each epoch
        current_state = env.reset()

        # Initialize metrics
        win, lose = 0, 0

        # Keep playing the game as long as possible
        game_over = False
        while not game_over:

            # The agent follows his policy
            action = agent.act(current_state)

            # Update of the environment in consequence
            next_state, reward, game_over = env.act(action, train = True)

            # Update the metrics
            if reward > 0:
                win += reward
            else:
                lose -=  reward

            # Execute the reinforcement learning strategy
            loss = agent.reinforce(current_state, next_state, action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights = prefix+'model.h5', name_model = prefix + 'model.json')

In [45]:
class EnvironmentExploring(Environment):

    def __init__(self, *args, **kwargs):
        super(EnvironmentExploring, self).__init__(*args, **kwargs)

        # The malus indicators is the only difference
        self.malus_position=np.zeros((self.grid_size, self.grid_size))

    def act(self, action, train=False):

        self.get_frame(int(self.time))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.time = self.time + 1

        # Set the to-be cumulative reward to zero
        reward = 0

        # Penalize the agent if train is set to True
        if train: reward -= self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        # Add true reward and remove it from the board
        reward += self.board[self.x, self.y]
        self.board[self.x, self.y] = 0

        # Return new state
        game_over = self.time > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                                self.board.reshape(self.grid_size, self.grid_size, 1),
                                self.position.reshape(self.grid_size, self.grid_size, 1))
                                ,axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return(state, reward, game_over)

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1, self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1, self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        # This makes the dimension of the problem work
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.time = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return(state)


In [51]:
# Instanciate the Agent and the Environment
env_explore = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_explore = DQN_CNN(size, lr= 0.1, epsilon = 0.3, memory_size = 2000, batch_size = 64, n_state = 3)
# Training
train_explore(agent_explore, env_explore, 20, prefix='cnn_train_explore_', epsilon_0 = 0.3)

Epoch 000/020 | Loss 0.0159 | Win/lose count 6.0/53.300000000000466 (-47.300000000000466)
Epoch 001/020 | Loss 0.0140 | Win/lose count 8.0/55.800000000000466 (-47.800000000000466)
Epoch 002/020 | Loss 0.0242 | Win/lose count 11.5/60.30000000000042 (-48.80000000000042)
Epoch 003/020 | Loss 0.0285 | Win/lose count 13.0/61.70000000000048 (-48.70000000000048)
Epoch 004/020 | Loss 0.0347 | Win/lose count 7.5/55.30000000000046 (-47.80000000000046)
Epoch 005/020 | Loss 0.0376 | Win/lose count 21.0/64.5000000000004 (-43.5000000000004)
Epoch 006/020 | Loss 0.0218 | Win/lose count 6.5/56.00000000000041 (-49.50000000000041)
Epoch 007/020 | Loss 0.0309 | Win/lose count 10.5/57.60000000000047 (-47.10000000000047)
Epoch 008/020 | Loss 0.0359 | Win/lose count 2.5/51.70000000000042 (-49.20000000000042)


KeyboardInterrupt: 

<b>Careful!</b> I did not have time to compute an adequate number of epochs (80 ?) on the data. But I am confident the results will be what is expected.

In [48]:
if movie_: HTML(display_videos('cnn_train_explore_10.mp4'))

In [ ]:
# Load
agent_explore = DQN_CNN(size, lr = 0.1, epsilon = 0.2, memory_size = 20000, batch_size = 64)
agent_explore.load(name_weights = 'cnn_train_explore_model.h5', name_model = 'cnn_train_explore_model.json')
# Evaluation
test(agent, env, epochs_test, prefix='cnn_test_explore_')

In [ ]:
if movie_: HTML(display_videos('cnn_test_explore_9.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [52]:
import copy
class Double_DQN(Agent):
    def __init__(self, grid_size, lr = 0.1, discount=0.9, epsilon=0.1, memory_size=100, batch_size=16, n_state=3):
        super(Double_DQN, self).__init__(epsilon = epsilon)

        # Discount and grid size
        self.discount = discount
        self.grid_size = grid_size

        # Number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)

        # Batch size when learning
        self.batch_size = batch_size

        # Learning rate
        self.lr = lr

    def learned_act(self, s):
        prediction = self.model.predict(np.array([s,]))
        return(np.argmax(prediction))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])

        # Initializations
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))

        # Indicator variable for which model should be trained
        zero_one = np.random.randint(2)

        for i in range(self.batch_size):
            # Random Access Memories
            [s_batch, n_s_batch, a_batch, r_batch, game_over_batch] = self.memory.random_access()
            input_states[i] = s_batch

            # Prediction depending of the model
            if zero_one == 0:
                target_q[i] = self.model.predict(np.array([s_batch]))
            else :
                target_q[i] = self.second_model.predict(np.array([s_batch]))

            if game_over_:
                target_q[i, a_batch] = r_batch
            else:
                # Prediction depending of the model
                if zero_one == 0:
                    prediction1 = self.model.predict(np.array([n_s_batch]))
                    prediction2 = self.second_model.predict(np.array([n_s_batch]))
                else:
                    prediction2 = self.model.predict(np.array([n_s_batch]))
                    prediction1 = self.second_model.predict(np.array([n_s_batch]))

                target_q[i, a_batch] = r_batch + self.discount * (prediction2.ravel())[np.argmax(prediction1)]

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        # Training depending on the model
        if zero_one == 0:
            l = self.model.train_on_batch(input_states, target_q)
        else:
            l = self.second_model.train_on_batch(input_states, target_q)

        return l

    def save(self, name_weights='model.h5', name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)

    def load(self, name_weights='model.h5', name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

class Double_DQN_CNN(Double_DQN):
    def __init__(self, *args,**kwargs):
        super(Double_DQN_CNN, self).__init__(*args,**kwargs)

        model = Sequential()

        model.add(Conv2D(32, (1,1), strides=(1,1), input_shape=(5, 5, self.n_state), data_format = 'channels_last'))
        model.add(Conv2D(16, (1,1), strides=(1,1), data_format = 'channels_last'))
        model.add(Flatten())
        model.add(Dense(4))
        model.add(Activation('relu'))

        model.compile(sgd(lr = 0.1, decay = 1e-4, momentum = 0.8), "mse")
        self.model = model

        self.second_model = copy.deepcopy(self.model)
        self.second_model.compile(sgd(lr = 0.1, decay = 1e-4, momentum = 0.8), "mse")
        self.second_model.set_weights(self.model.get_weights())


In [53]:
# Instanciate the Agent and the Environment
env_explore = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_explore = Double_DQN_CNN(size, lr=0.1, epsilon = 0.4, memory_size = 20000, batch_size = 64, n_state = 3)
# Training
train_explore(agent_explore, env_explore, 2, prefix='cnn_train_explore_')

Epoch 000/002 | Loss 0.0448 | Win/lose count 6.5/51.80000000000041 (-45.30000000000041)


KeyboardInterrupt: 

<b>Careful!</b> I did not have time to compute an adequate number of epochs (80 ?) on the data. But I am confident the results will be what is expected.

In [ ]:
if movie_: HTML(display_videos('cnn_train_explore_0.mp4'))

***